In [1]:
# !pip install -U scikit-learn
import pandas as pd
import numpy as np

from platform import python_version
import numpy as np
import geopandas as gpd
from shapely.geometry import shape
import matplotlib.pyplot as plt
import tensorflow as tf
import keras_tuner as kt
import sklearn as skl
from sklearn.datasets import make_circles


pd.set_option('display.max_columns', 100)

%precision 2
import pandas as pd
from datetime import datetime
%matplotlib inline


import os
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, RobustScaler
from sklearn.compose import ColumnTransformer

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
%matplotlib inline



# Initial imports
import pandas as pd
from pathlib import Path
from sklearn import tree
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import accuracy_score, r2_score, recall_score, precision_score, roc_curve, roc_auc_score, f1_score
from sklearn.metrics import classification_report, confusion_matrix

# Needed for decision tree visualization
import pydotplus
from IPython.display import Image


# Start a SparkSession
import findspark
findspark.init()

python 3.10.9
numpy 1.23.5
pandas 1.5.3
klib 1.0.7


In [2]:
df = pd.read_csv('Resources/2020_Clean_Traffic_Crashes.csv')

In [3]:
df

,RD_NO,CRASH_DATE,POSTED_SPEED_LIMIT,TRAFFIC_CONTROL_DEVICE,DEVICE_CONDITION,WEATHER_CONDITION,LIGHTING_CONDITION,FIRST_CRASH_TYPE,TRAFFICWAY_TYPE,ALIGNMENT,ROADWAY_SURFACE_COND,ROAD_DEFECT,CRASH_TYPE,DAMAGE,PRIM_CONTRIBUTORY_CAUSE,STREET_NO,STREET_DIRECTION,STREET_NAME,WORK_ZONE_TYPE,NUM_UNITS,MOST_SEVERE_INJURY,INJURIES_TOTAL,INJURIES_FATAL,INJURIES_INCAPACITATING,CRASH_HOUR,CRASH_DAY_OF_WEEK,CRASH_MONTH,LATITUDE,LONGITUDE,CRASH_YEAR,INJURY_CAT,DAMAGE_COST,year
0,JD184955,2020-03-13 21:45:00,25,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",REAR END,ONE-WAY,STRAIGHT AND LEVEL,DRY,NO DEFECTS,NO INJURY / DRIVE AWAY,$500 OR LESS,FOLLOWING TOO CLOSELY,183,S,WELLS ST,NaN,2.0,NO INDICATION OF INJURY,0.0,0.0,0.0,21,6,3,41.879415,-87.633606,2020,0,500,2020
1,JD116085,2020-01-14 23:15:00,30,STOP SIGN/FLASHER,FUNCTIONING PROPERLY,CLEAR,"DARKNESS, LIGHTED ROAD",TURNING,FOUR WAY,STRAIGHT AND LEVEL,DRY,NO DEFECTS,NO INJURY / DRIVE AWAY,"$501 - $1,500",IMPROPER TURNING/NO SIGNAL,1214,E,53RD ST,NaN,2.0,NO INDICATION OF INJURY,0.0,0.0,0.0,23,3,1,41.799572,-87.595997,2020,0,1000,2020
2,JD122763,2020-01-20 22:48:00,35,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",ANGLE,UNKNOWN INTERSECTION TYPE,STRAIGHT AND LEVEL,DRY,UNKNOWN,NO INJURY / DRIVE AWAY,"OVER $1,500",DISREGARDING STOP SIGN,3424,W,OGDEN AVE,NaN,2.0,NO INDICATION OF INJURY,0.0,0.0,0.0,22,2,1,41.856015,-87.711318,2020,0,1500,2020
3,JD134580,2020-01-30 22:54:00,30,UNKNOWN,UNKNOWN,UNKNOWN,"DARKNESS, LIGHTED ROAD",REAR TO FRONT,UNKNOWN INTERSECTION TYPE,STRAIGHT AND LEVEL,UNKNOWN,UNKNOWN,NO INJURY / DRIVE AWAY,"OVER $1,500",NOT APPLICABLE,3700,W,CONGRESS PKWY,NaN,2.0,NO INDICATION OF INJURY,0.0,0.0,0.0,22,5,1,41.874595,-87.718136,2020,0,1500,2020
4,JD159847,2020-02-20 19:50:00,20,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",ANGLE,T-INTERSECTION,STRAIGHT AND LEVEL,DRY,NO DEFECTS,NO INJURY / DRIVE AWAY,"$501 - $1,500",FAILING TO REDUCE SPEED TO AVOID CRASH,4164,W,WABANSIA AVE,NaN,2.0,NO INDICATION OF INJURY,0.0,0.0,0.0,19,5,2,41.911754,-87.731212,2020,0,1000,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24366,JD189551,2020-03-13 14:50:00,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,DAYLIGHT,TURNING,T-INTERSECTION,STRAIGHT AND LEVEL,DRY,NO DEFECTS,NO INJURY / DRIVE AWAY,"OVER $1,500",FOLLOWING TOO CLOSELY,101,W,87TH ST,NaN,2.0,NO INDICATION OF INJURY,0.0,0.0,0.0,14,6,3,41.736154,-87.627240,2020,0,1500,2020
24367,JD149870,2020-02-13 00:05:00,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,SNOW,DARKNESS,REAR END,NOT DIVIDED,STRAIGHT AND LEVEL,SNOW OR SLUSH,NO DEFECTS,INJURY AND / OR TOW DUE TO CRASH,"OVER $1,500",NOT APPLICABLE,13000,S,STONY ISLAND AVE,NaN,2.0,NO INDICATION OF INJURY,0.0,0.0,0.0,0,5,2,0.000000,0.000000,2020,1,1500,2020
24368,JD144014,2020-02-07 20:23:00,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,"DARKNESS, LIGHTED ROAD",REAR END,NOT DIVIDED,STRAIGHT AND LEVEL,UNKNOWN,UNKNOWN,NO INJURY / DRIVE AWAY,"$501 - $1,500",UNABLE TO DETERMINE,815,S,AUSTIN BLVD,NaN,2.0,NO INDICATION OF INJURY,0.0,0.0,0.0,20,6,2,0.000000,0.000000,2020,0,1000,2020
24369,JD118213,2020-01-16 17:30:00,30,UNKNOWN,FUNCTIONING PROPERLY,CLEAR,DUSK,REAR END,DIVIDED - W/MEDIAN (NOT RAISED),STRAIGHT AND LEVEL,DRY,NO DEFECTS,NO INJURY / DRIVE AWAY,"$501 - $1,500",FOLLOWING TOO CLOSELY,502,W,OHARE ST,NaN,2.0,NO INDICATION OF INJURY,0.0,0.0,0.0,17,5,1,0.000000,0.000000,2020,0,1000,2020


In [4]:
df.CRASH_TYPE.value_counts()

NO INJURY / DRIVE AWAY              17606
INJURY AND / OR TOW DUE TO CRASH     6765
Name: CRASH_TYPE, dtype: int64

In [5]:
# retrieving list of numeric columns 
num_cols = df.select_dtypes('number').columns.tolist()
num_cols

['POSTED_SPEED_LIMIT',
 'STREET_NO',
 'NUM_UNITS',
 'INJURIES_TOTAL',
 'INJURIES_FATAL',
 'INJURIES_INCAPACITATING',
 'CRASH_HOUR',
 'CRASH_DAY_OF_WEEK',
 'CRASH_MONTH',
 'LATITUDE',
 'LONGITUDE',
 'CRASH_YEAR',
 'INJURY_CAT',
 'DAMAGE_COST',
 'year']

In [6]:
# Encoding catagory variables
# retrieving list of categorical columns 
cat_cols = df.select_dtypes('object').columns.tolist()
cat_cols

['RD_NO',
 'CRASH_DATE',
 'TRAFFIC_CONTROL_DEVICE',
 'DEVICE_CONDITION',
 'WEATHER_CONDITION',
 'LIGHTING_CONDITION',
 'FIRST_CRASH_TYPE',
 'TRAFFICWAY_TYPE',
 'ALIGNMENT',
 'ROADWAY_SURFACE_COND',
 'ROAD_DEFECT',
 'CRASH_TYPE',
 'DAMAGE',
 'PRIM_CONTRIBUTORY_CAUSE',
 'STREET_DIRECTION',
 'STREET_NAME',
 'WORK_ZONE_TYPE',
 'MOST_SEVERE_INJURY']

In [7]:
# Scaling the numeric columns
crash_data_scaled = StandardScaler().fit_transform(df[['POSTED_SPEED_LIMIT',
 'STREET_NO',
 'NUM_UNITS',
 'INJURIES_TOTAL',
 'INJURIES_FATAL',
 'INJURIES_INCAPACITATING',
 'CRASH_HOUR',
 'CRASH_DAY_OF_WEEK',
 'CRASH_MONTH',
 'LATITUDE',
 'LONGITUDE',
 'CRASH_YEAR',
 'INJURY_CAT',
 'year']])

# Creating a DataFrame with with the scaled data
df_crash_transformed = pd.DataFrame(crash_data_scaled, columns=['POSTED_SPEED_LIMIT',
 'STREET_NO',
 'NUM_UNITS',
 'INJURIES_TOTAL',
 'INJURIES_FATAL',
 'INJURIES_INCAPACITATING',
 'CRASH_HOUR',
 'CRASH_DAY_OF_WEEK',
 'CRASH_MONTH',
 'LATITUDE',
 'LONGITUDE',
 'CRASH_YEAR',
 'INJURY_CAT',
 'year'])

# Display sample data
df_crash_transformed.head()

,POSTED_SPEED_LIMIT,STREET_NO,NUM_UNITS,INJURIES_TOTAL,INJURIES_FATAL,INJURIES_INCAPACITATING,CRASH_HOUR,CRASH_DAY_OF_WEEK,CRASH_MONTH,LATITUDE,LONGITUDE,CRASH_YEAR,INJURY_CAT,year
0,-0.598819,-1.220573,-0.070527,-0.329705,-0.027324,-0.126388,1.446452,0.925862,1.371172,0.102790,-0.091939,0.0,-0.619874,0.0
1,0.261615,-0.860676,-0.070527,-0.329705,-0.027324,-0.126388,1.807973,-0.617481,-1.162422,0.083023,-0.087493,0.0,-0.619874,0.0
2,1.122048,-0.089219,-0.070527,-0.329705,-0.027324,-0.126388,1.627213,-1.131928,-1.162422,0.096996,-0.101125,0.0,-0.619874,0.0
3,0.261615,0.007126,-0.070527,-0.329705,-0.027324,-0.126388,1.627213,0.411415,-1.162422,0.101596,-0.101931,0.0,-0.619874,0.0
4,-1.459252,0.169097,-0.070527,-0.329705,-0.027324,-0.126388,1.084932,0.411415,0.104375,0.110796,-0.103477,0.0,-0.619874,0.0


In [8]:
# Encoding catagory variables
# retrieving list of categorical columns 
cat_cols = df.select_dtypes('object').columns.tolist()
cat_cols

['RD_NO',
 'CRASH_DATE',
 'TRAFFIC_CONTROL_DEVICE',
 'DEVICE_CONDITION',
 'WEATHER_CONDITION',
 'LIGHTING_CONDITION',
 'FIRST_CRASH_TYPE',
 'TRAFFICWAY_TYPE',
 'ALIGNMENT',
 'ROADWAY_SURFACE_COND',
 'ROAD_DEFECT',
 'CRASH_TYPE',
 'DAMAGE',
 'PRIM_CONTRIBUTORY_CAUSE',
 'STREET_DIRECTION',
 'STREET_NAME',
 'WORK_ZONE_TYPE',
 'MOST_SEVERE_INJURY']

In [9]:
df_crash_dummies = pd.get_dummies(df[['RD_NO',
 'CRASH_DATE',
 'TRAFFIC_CONTROL_DEVICE',
 'DEVICE_CONDITION',
 'WEATHER_CONDITION',
 'LIGHTING_CONDITION',
 'FIRST_CRASH_TYPE',
 'TRAFFICWAY_TYPE',
 'ALIGNMENT',
 'ROADWAY_SURFACE_COND',
 'ROAD_DEFECT',
 'CRASH_TYPE',
 'DAMAGE',
 'PRIM_CONTRIBUTORY_CAUSE',
 'STREET_DIRECTION',
 'STREET_NAME',
 'WORK_ZONE_TYPE',
 'MOST_SEVERE_INJURY']])

In [10]:
# Display sample data
df_crash_dummies.head()

,RD_NO_DJ166726,RD_NO_DJ177595,RD_NO_HD114501,RD_NO_JB115285,RD_NO_JB135782,RD_NO_JB137837,RD_NO_JB140557,RD_NO_JB140875,RD_NO_JB151260,RD_NO_JB174997,RD_NO_JB184588,RD_NO_JB189494,RD_NO_JC100035,RD_NO_JC100287,RD_NO_JC100614,RD_NO_JC100628,RD_NO_JC100678,RD_NO_JC100935,RD_NO_JC101015,RD_NO_JC101356,RD_NO_JC101578,RD_NO_JC101767,RD_NO_JC101780,RD_NO_JC102097,RD_NO_JC102125,RD_NO_JC102158,RD_NO_JC102296,RD_NO_JC102526,RD_NO_JC102606,RD_NO_JC102675,RD_NO_JC102806,RD_NO_JC102847,RD_NO_JC102869,RD_NO_JC102985,RD_NO_JC103157,RD_NO_JC103244,RD_NO_JC103329,RD_NO_JC103537,RD_NO_JC103539,RD_NO_JC103893,RD_NO_JC104045,RD_NO_JC104077,RD_NO_JC104518,RD_NO_JC104646,RD_NO_JC104978,RD_NO_JC105255,RD_NO_JC105261,RD_NO_JC105328,RD_NO_JC105353,RD_NO_JC105387,...,STREET_NAME_WALTON ST,STREET_NAME_WARNER AVE,STREET_NAME_WARREN BLVD,STREET_NAME_WARWICK AVE,STREET_NAME_WASHBURNE AVE,STREET_NAME_WASHINGTON BLVD,STREET_NAME_WASHINGTON ST,STREET_NAME_WASHTENAW AVE,STREET_NAME_WAVELAND AVE,STREET_NAME_WAYMAN ST,STREET_NAME_WAYNE AVE,STREET_NAME_WEBSTER AVE,STREET_NAME_WELLINGTON AVE,STREET_NAME_WELLS ST,STREET_NAME_WENTWORTH AVE,STREET_NAME_WEST END AVE,STREET_NAME_WESTERN AVE,STREET_NAME_WESTERN BLVD,STREET_NAME_WESTSHORE DR,STREET_NAME_WHIPPLE ST,STREET_NAME_WIELAND ST,STREET_NAME_WILCOX ST,STREET_NAME_WILLOW ST,STREET_NAME_WILSON AVE,STREET_NAME_WILSON DR,STREET_NAME_WINCHESTER AVE,STREET_NAME_WINDSOR AVE,STREET_NAME_WINNEMAC AVE,STREET_NAME_WINONA ST,STREET_NAME_WINSTON AVE,STREET_NAME_WINTHROP AVE,STREET_NAME_WISCONSIN ST,STREET_NAME_WOLCOTT AVE,STREET_NAME_WOLF POINT PLZ,STREET_NAME_WOLFRAM ST,STREET_NAME_WOOD ST,STREET_NAME_WOODLAWN AVE,STREET_NAME_WRIGHTWOOD AVE,STREET_NAME_YALE AVE,STREET_NAME_YATES BLVD,STREET_NAME_ZEMKE RD,WORK_ZONE_TYPE_CONSTRUCTION,WORK_ZONE_TYPE_MAINTENANCE,WORK_ZONE_TYPE_UNKNOWN,WORK_ZONE_TYPE_UTILITY,MOST_SEVERE_INJURY_FATAL,MOST_SEVERE_INJURY_INCAPACITATING INJURY,MOST_SEVERE_INJURY_NO INDICATION OF INJURY,MOST_SEVERE_INJURY_NONINCAPACITATING INJURY,"MOST_SEVERE_INJURY_REPORTED, NOT EVIDENT"
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [11]:
# Concatenate the `df_crash_transformed` encoded dummies with the scaled data DataFrame
final_crash_scaled = pd.concat([df_crash_transformed,df_crash_dummies], axis=1)

# Display the sample data
final_crash_scaled.head()

,POSTED_SPEED_LIMIT,STREET_NO,NUM_UNITS,INJURIES_TOTAL,INJURIES_FATAL,INJURIES_INCAPACITATING,CRASH_HOUR,CRASH_DAY_OF_WEEK,CRASH_MONTH,LATITUDE,LONGITUDE,CRASH_YEAR,INJURY_CAT,year,RD_NO_DJ166726,RD_NO_DJ177595,RD_NO_HD114501,RD_NO_JB115285,RD_NO_JB135782,RD_NO_JB137837,RD_NO_JB140557,RD_NO_JB140875,RD_NO_JB151260,RD_NO_JB174997,RD_NO_JB184588,RD_NO_JB189494,RD_NO_JC100035,RD_NO_JC100287,RD_NO_JC100614,RD_NO_JC100628,RD_NO_JC100678,RD_NO_JC100935,RD_NO_JC101015,RD_NO_JC101356,RD_NO_JC101578,RD_NO_JC101767,RD_NO_JC101780,RD_NO_JC102097,RD_NO_JC102125,RD_NO_JC102158,RD_NO_JC102296,RD_NO_JC102526,RD_NO_JC102606,RD_NO_JC102675,RD_NO_JC102806,RD_NO_JC102847,RD_NO_JC102869,RD_NO_JC102985,RD_NO_JC103157,RD_NO_JC103244,...,STREET_NAME_WALTON ST,STREET_NAME_WARNER AVE,STREET_NAME_WARREN BLVD,STREET_NAME_WARWICK AVE,STREET_NAME_WASHBURNE AVE,STREET_NAME_WASHINGTON BLVD,STREET_NAME_WASHINGTON ST,STREET_NAME_WASHTENAW AVE,STREET_NAME_WAVELAND AVE,STREET_NAME_WAYMAN ST,STREET_NAME_WAYNE AVE,STREET_NAME_WEBSTER AVE,STREET_NAME_WELLINGTON AVE,STREET_NAME_WELLS ST,STREET_NAME_WENTWORTH AVE,STREET_NAME_WEST END AVE,STREET_NAME_WESTERN AVE,STREET_NAME_WESTERN BLVD,STREET_NAME_WESTSHORE DR,STREET_NAME_WHIPPLE ST,STREET_NAME_WIELAND ST,STREET_NAME_WILCOX ST,STREET_NAME_WILLOW ST,STREET_NAME_WILSON AVE,STREET_NAME_WILSON DR,STREET_NAME_WINCHESTER AVE,STREET_NAME_WINDSOR AVE,STREET_NAME_WINNEMAC AVE,STREET_NAME_WINONA ST,STREET_NAME_WINSTON AVE,STREET_NAME_WINTHROP AVE,STREET_NAME_WISCONSIN ST,STREET_NAME_WOLCOTT AVE,STREET_NAME_WOLF POINT PLZ,STREET_NAME_WOLFRAM ST,STREET_NAME_WOOD ST,STREET_NAME_WOODLAWN AVE,STREET_NAME_WRIGHTWOOD AVE,STREET_NAME_YALE AVE,STREET_NAME_YATES BLVD,STREET_NAME_ZEMKE RD,WORK_ZONE_TYPE_CONSTRUCTION,WORK_ZONE_TYPE_MAINTENANCE,WORK_ZONE_TYPE_UNKNOWN,WORK_ZONE_TYPE_UTILITY,MOST_SEVERE_INJURY_FATAL,MOST_SEVERE_INJURY_INCAPACITATING INJURY,MOST_SEVERE_INJURY_NO INDICATION OF INJURY,MOST_SEVERE_INJURY_NONINCAPACITATING INJURY,"MOST_SEVERE_INJURY_REPORTED, NOT EVIDENT"
0,-0.598819,-1.220573,-0.070527,-0.329705,-0.027324,-0.126388,1.446452,0.925862,1.371172,0.102790,-0.091939,0.0,-0.619874,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,0.261615,-0.860676,-0.070527,-0.329705,-0.027324,-0.126388,1.807973,-0.617481,-1.162422,0.083023,-0.087493,0.0,-0.619874,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,1.122048,-0.089219,-0.070527,-0.329705,-0.027324,-0.126388,1.627213,-1.131928,-1.162422,0.096996,-0.101125,0.0,-0.619874,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,0.261615,0.007126,-0.070527,-0.329705,-0.027324,-0.126388,1.627213,0.411415,-1.162422,0.101596,-0.101931,0.0,-0.619874,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,-1.459252,0.169097,-0.070527,-0.329705,-0.027324,-0.126388,1.084932,0.411415,0.104375,0.110796,-0.103477,0.0,-0.619874,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [12]:
# Split the data into X (features) and y (target)
# The y variable should focus on the target column
y = final_crash_scaled['MOST_SEVERE_INJURY_NO INDICATION OF INJURY'].ravel()

# The X variable should include all features except the target
X = final_crash_scaled.drop(columns=['MOST_SEVERE_INJURY_NO INDICATION OF INJURY'])

In [13]:
y[:10]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=uint8)

In [14]:
# Split into testing and training sets using train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y)
X_train

,POSTED_SPEED_LIMIT,STREET_NO,NUM_UNITS,INJURIES_TOTAL,INJURIES_FATAL,INJURIES_INCAPACITATING,CRASH_HOUR,CRASH_DAY_OF_WEEK,CRASH_MONTH,LATITUDE,LONGITUDE,CRASH_YEAR,INJURY_CAT,year,RD_NO_DJ166726,RD_NO_DJ177595,RD_NO_HD114501,RD_NO_JB115285,RD_NO_JB135782,RD_NO_JB137837,RD_NO_JB140557,RD_NO_JB140875,RD_NO_JB151260,RD_NO_JB174997,RD_NO_JB184588,RD_NO_JB189494,RD_NO_JC100035,RD_NO_JC100287,RD_NO_JC100614,RD_NO_JC100628,RD_NO_JC100678,RD_NO_JC100935,RD_NO_JC101015,RD_NO_JC101356,RD_NO_JC101578,RD_NO_JC101767,RD_NO_JC101780,RD_NO_JC102097,RD_NO_JC102125,RD_NO_JC102158,RD_NO_JC102296,RD_NO_JC102526,RD_NO_JC102606,RD_NO_JC102675,RD_NO_JC102806,RD_NO_JC102847,RD_NO_JC102869,RD_NO_JC102985,RD_NO_JC103157,RD_NO_JC103244,...,STREET_NAME_WALNUT ST,STREET_NAME_WALTON ST,STREET_NAME_WARNER AVE,STREET_NAME_WARREN BLVD,STREET_NAME_WARWICK AVE,STREET_NAME_WASHBURNE AVE,STREET_NAME_WASHINGTON BLVD,STREET_NAME_WASHINGTON ST,STREET_NAME_WASHTENAW AVE,STREET_NAME_WAVELAND AVE,STREET_NAME_WAYMAN ST,STREET_NAME_WAYNE AVE,STREET_NAME_WEBSTER AVE,STREET_NAME_WELLINGTON AVE,STREET_NAME_WELLS ST,STREET_NAME_WENTWORTH AVE,STREET_NAME_WEST END AVE,STREET_NAME_WESTERN AVE,STREET_NAME_WESTERN BLVD,STREET_NAME_WESTSHORE DR,STREET_NAME_WHIPPLE ST,STREET_NAME_WIELAND ST,STREET_NAME_WILCOX ST,STREET_NAME_WILLOW ST,STREET_NAME_WILSON AVE,STREET_NAME_WILSON DR,STREET_NAME_WINCHESTER AVE,STREET_NAME_WINDSOR AVE,STREET_NAME_WINNEMAC AVE,STREET_NAME_WINONA ST,STREET_NAME_WINSTON AVE,STREET_NAME_WINTHROP AVE,STREET_NAME_WISCONSIN ST,STREET_NAME_WOLCOTT AVE,STREET_NAME_WOLF POINT PLZ,STREET_NAME_WOLFRAM ST,STREET_NAME_WOOD ST,STREET_NAME_WOODLAWN AVE,STREET_NAME_WRIGHTWOOD AVE,STREET_NAME_YALE AVE,STREET_NAME_YATES BLVD,STREET_NAME_ZEMKE RD,WORK_ZONE_TYPE_CONSTRUCTION,WORK_ZONE_TYPE_MAINTENANCE,WORK_ZONE_TYPE_UNKNOWN,WORK_ZONE_TYPE_UTILITY,MOST_SEVERE_INJURY_FATAL,MOST_SEVERE_INJURY_INCAPACITATING INJURY,MOST_SEVERE_INJURY_NONINCAPACITATING INJURY,"MOST_SEVERE_INJURY_REPORTED, NOT EVIDENT"
5545,-2.319685,1.054353,-0.070527,-0.329705,-0.027324,-0.126388,1.265692,-1.131928,1.371172,0.076518,-0.085786,0.0,-0.619874,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8841,0.261615,-1.210101,-0.070527,-0.329705,-0.027324,-0.126388,-0.903430,-0.103033,-1.162422,0.105774,-0.092062,0.0,-0.619874,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6518,0.261615,0.161069,-0.070527,-0.329705,-0.027324,-0.126388,0.181131,0.411415,-1.162422,0.121133,-0.103434,0.0,-0.619874,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1701,-3.180118,-0.253982,-0.070527,-0.329705,-0.027324,-0.126388,0.000371,-1.131928,1.371172,0.116708,-0.097807,0.0,-0.619874,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
15622,0.261615,-0.960861,-0.070527,-0.329705,-0.027324,-0.126388,-1.807231,0.411415,1.371172,0.047773,-0.087902,0.0,-0.619874,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20091,0.261615,0.235771,-0.070527,-0.329705,-0.027324,-0.126388,0.723412,1.44031

# Compile, Train and Evaluate the Model

In [15]:
# Splitting into Train and Test sets
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state = 42)

In [18]:
#Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len( X_train[0])
# hidden_nodes_layer1=7
# hidden_nodes_layer2=14
# hidden_nodes_layer3=21

# nn = tf.keras.models.Sequential()

# # First hidden layer
# nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation='relu'))

# # Second hidden layer
# nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation='relu'))

# # Output layer
# nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# # Check the structure of the model
# nn.summary()

KeyError: 0

# Making Predictions Using the Random Forest Model